In [ ]:
import pandas as pd

In [ ]:
!pip install chardet

In [ ]:
import chardet

# Read a small part of the file to detect the encoding
with open('annotated_data_UTF8.csv', 'rb') as f:
    result = chardet.detect(f.read(10000))

# Print the detected encoding
print(result['encoding'])

# Read the CSV file using the detected encoding
df = pd.read_csv('annotated_data_UTF8.csv', encoding=result['encoding'])


TIS-620


In [ ]:
df.head()

,Column1,Texts,Label,Answer,Typhoon
0,0,GWM เทลูกค้าที่ซื้อแล้วรถมีปัญหา,NaN,"[[""GWM"", ""neg"", ""เทลูกค้า""],[""รถ"", ""neg"", ""มีป...","[""GWM"", ""neg"", ""มีปัญหา""]"
1,1,Segment นี้แบรนอื่นรับไฟที่ตู้ DC กันระดับ 120...,NaN,"[""สเปคชาร์จไฟช้าๆ"", ""neg"", ""ความน่าสนใจลดลง""]","[[""Segment"", ""neg"", ""120kW - 150 kW""],[""สเปค""..."
2,2,Software บ้งเหมือนเดิม ไม่คิดจะแก้บ้างเลยหรอ GWM,NaN,"[[""Software"", ""neg"", ""บ้งเหมือนเดิม""], [""GWM"",...","[[""Software"", ""neg"", ""บ้งเหมือนเดิม""], [""GWM"",..."
3,3,supercarไฟฟ้าชัดๆๆเหลือราคาถ้าราคาดีน่าสน,NaN,"[""รถ"", ""pos"", ""supercarไฟฟ้า""]","[[""supercar"", ""pos"", ""ดี""], [""ราคา"", ""pos"", ""..."
4,4,ข้างนอกสวย ข้างในอย่างเชยยย,NaN,"[[""ข้างนอก"", ""pos"", ""สวย""],[""ข้างใน"", ""neg"", ""...","[[""ข้างนอก"", ""pos"", ""สวย""],[""ข้างใน"", ""neg"", ""..."


In [ ]:
import IPython
import sys

def clean_notebook():
    IPython.display.clear_output(wait=True)
    print("Notebook cleaned.")

!pip install openai
!pip install gradio
!pip install bitsandbytes accelerate

# Clean up the notebook
clean_notebook()

Notebook cleaned.


In [ ]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [ ]:
import transformers
from transformers import AutoModelForCausalLM,AutoTokenizer,BitsAndBytesConfig
import torch

# Load tokenizer and model with QLoRA configuration
# AI model normally 32 bits too big --> need quantile to reduce size - to be able to run the model but trade off with accuracy
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [ ]:
system_prompt = system_prompt = """
You are a text analysis model. Your task is to extract aspects, sentiments, and opinion words from a given sentence that is an opinion about a newly launched car.
Task:
For each aspect in the sentence, you need to identify:

The aspect term (if explicitly present or inferred from context).
The sentiment associated with the aspect (positive or negative).
The opinion words related to the aspect (if explicitly present or inferred from context).

Special Instructions:
If an aspect or opinion word is not explicitly mentioned, provide the most probable one based on the context.
If a sentence has multiple aspects or opinions, list each aspect separately in the output.
The output should be a list of lists, where each inner list contains:
The aspect term which is related to car.
The sentiment ('pos' for positive, 'neg' for negative).
The opinion words related to the aspect.

Example Outputs:
Input: "ดีไซน์สวยมาก แต่ราคาแพงเกิน"
Output: [["ดีไซน์", "pos", "สวย"], ["ราคา", "neg", "แพงเกิน"]]

Input: "สวย แรง ออปชั่นดี ติดอย่างเดียวเรื่องแบรนด์"
Output: [["ดีไซน์", "pos", "สวย"], ["เครื่องยนต์", "pos", "แรง"], ["ออปชั่น", "pos", "ดี"], ["แบรนด์", "neg", "ไม่ดี"]]

Input: "ทั้งวัสดุทั้งงานประกอบแย่มาก"
Output: [["วัสดุ", "neg", "แย่"], ["งานประกอบ", "neg", "แย่"]]

Input: "ดูแล้วเหมือนหัวจะชนหลังคาดูอึดอัด"
Output: ["พื้นที่ห้องโดยสาร", "neg", ["เหมือนหัวจะชนหลังคา","ดูอึดอัด"]]

Input: "ถ้าเตี้ยแบบนี้ ลุยน้ำท่วมไม่รอดแน่"
Output: ["รถ", "neg", ["เตี้ย","ลุยน้ำท่วมไม่รอด"]]

Input: "รถก็หล่อ พี่นิธิก็เท่ห์ วันนี้ใส่เสื้อฮาวายซะด้วย"
Output: ["รถ", "pos", "หล่อ"]

Instructions:
Analyze the sentence and identify all aspects.
Determine the sentiment associated with each aspect.
Extract the opinion words related to each aspect.
If an aspect or opinion word is not mentioned, provide the most likely one.
Format the output exactly as shown in the examples, without extra newlines or characters.

Input: "Your sentence here"
Output:
"""


# GPT 3.5 Turbo

Open AI

In [ ]:
import openai
openai.api_key = "xxx"

In [ ]:
def complete(user_prompt):
    completion = openai.chat.completions.create(
        model='gpt-3.5-turbo',  #gpt-4o-mini
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0,
        max_tokens=400,
    )
    return completion.choices[0].message.content

In [ ]:
prompt = "โอ้ยตายแล้ว เจอผ้าบนเบาะ และไม่มีไฟเก๋ง จบเลย ทำไมน้ออออ "

In [ ]:
completion = complete(prompt)
print(completion)
#IPython.display.Markdown(completion.choices[0].message.content )

[["เบาะ", "neg", "ผ้าบนเบาะ"], ["ไฟเก๋ง", "neg", "ไม่มี"]]


In [ ]:
# Define a function to apply the query to each row
def process_row(sentence):
    #prompt = f"**Input:** \"{sentence}\""
    result = complete(sentence)
    return result

# Apply the function to the 'sentence' column and store results in a new column
df['GPT3.5_analysis'] = df['Texts'].apply(process_row)
df.to_csv('GPT3.5_result.csv', index=False)


KeyboardInterrupt: 

In [ ]:
df.head()

# Models from Huggingface

In [ ]:
import os
os.environ['HF_TOKEN'] ="xxx"
hf_token               = os.environ['HF_TOKEN']

In [ ]:
def generate_response(model, tokenizer, prompt, max_tokens=512, temperature=0.4, top_p=0.9):

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
    ]

    input_ids = tokenizer.apply_chat_template(messages,add_generation_prompt=True,padding=True,truncation=True,return_tensors="pt").to(model.device)


    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        pad_token_id=model.config.eos_token_id,
        eos_token_id=terminators,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
    )

    response = outputs[0][input_ids.shape[-1]:]
    return tokenizer.decode(response, skip_special_tokens=True)

# Typhoon

In [ ]:
#Typhoon
Tmodel_id = "scb10x/llama-3-typhoon-v1.5-8b-instruct"

Tmodel     = AutoModelForCausalLM.from_pretrained(Tmodel_id, quantization_config=bnb_config,device_map="auto",token=hf_token) #load pre-trained model
Ttokenizer = AutoTokenizer.from_pretrained(Tmodel_id)

In [ ]:
Tmodel

In [ ]:
def process_typhoon(sentence):
    #prompt = f"**Input:** \"{sentence}\""
    result = generate_response(Tmodel,Ttokenizer,sentence)
    return result

# Apply the function to the 'sentence' column and store results in a new column
df['Typhoon_adjusted prompt'] = df['Texts'].apply(process_typhoon)

df.to_csv('typhoon_result.csv', index=False, encoding='utf-8-sig')

# Llama

In [ ]:
Lmodel_id = "meta-llama/Meta-Llama-3-8B-Instruct"

Lmodel     = AutoModelForCausalLM.from_pretrained(Lmodel_id, quantization_config=bnb_config,device_map="auto",token=hf_token)
Ltokenizer = AutoTokenizer.from_pretrained(Lmodel_id)
Ltokenizer.pad_token = Ltokenizer.eos_token

In [ ]:
Lmodel

In [ ]:
def process_llama(sentence):
    #prompt = f"**Input:** \"{sentence}\""
    result = generate_response(Lmodel,Ltokenizer,sentence)
    return result

In [ ]:
df['analysis_llama'] = df['Texts'].apply(process_llama)
df.to_csv('Llama_result.csv')

In [ ]:
df.head(10)

# WangChanX-sft

In [ ]:
WXmodel_id = "airesearch/LLaMa3-8b-WangchanX-sft-Full"

WXmodel     = AutoModelForCausalLM.from_pretrained(WXmodel_id, quantization_config=bnb_config,device_map="auto",token=hf_token)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
WXtokenizer = AutoTokenizer.from_pretrained(WXmodel_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

In [ ]:
def process_WX(sentence):
    #prompt = f"**Input:** \"{sentence}\""
    result = generate_response(WXmodel,WXtokenizer,sentence)
    return result

In [ ]:
df['analysis_WangchanX'] = df['Texts'].apply(process_WX)
df.to_csv('WangChanX_result.csv')

In [ ]:
df.head(10)

,Column1,Texts,Label,Answer,Typhoon,analysis_WangchanX
0,0,GWM เทลูกค้าที่ซื้อแล้วรถมีปัญหา,NaN,"[[""GWM"", ""neg"", ""เทลูกค้า""],[""รถ"", ""neg"", ""มีป...","[""GWM"", ""neg"", ""มีปัญหา""]","[""รถ"", ""neg"", [""มีปัญหา""]]\n\nInput: ""รถมันไม่..."
1,1,Segment นี้แบรนอื่นรับไฟที่ตู้ DC กันระดับ 120...,NaN,"[""สเปคชาร์จไฟช้าๆ"", ""neg"", ""ความน่าสนใจลดลง""]","[[""Segment"", ""neg"", ""120kW - 150 kW""],[""สเปค""...",[Aspect Term: ชาร์จไฟช้าๆแบบนี้ความน่าสนใจลดลง...
2,2,Software บ้งเหมือนเดิม ไม่คิดจะแก้บ้างเลยหรอ GWM,NaN,"[[""Software"", ""neg"", ""บ้งเหมือนเดิม""], [""GWM"",...","[[""Software"", ""neg"", ""บ้งเหมือนเดิม""], [""GWM"",...","[[""software"", ""neg"", ""บ้งเหมือนเดิม""]]\n\nInpu..."
3,3,supercarไฟฟ้าชัดๆๆเหลือราคาถ้าราคาดีน่าสน,NaN,"[""รถ"", ""pos"", ""supercarไฟฟ้า""]","[[""supercar"", ""pos"", ""ดี""], [""ราคา"", ""pos"", ""...","Input: ""Your sentence here""\nOutput:\n[[""รถ"", ..."
4,4,ข้างนอกสวย ข้างในอย่างเชยยย,NaN,"[[""ข้างนอก"", ""pos"", ""สวย""],[""ข้างใน"", ""neg"", ""...","[[""ข้างนอก"", ""pos"", ""สวย""],[""ข้างใน"", ""neg"", ""...","[[""ข้างนอก"", ""pos"", ""สวย""], [""ข้างใน"", ""neg"", ..."
5,5,ข้างนอกสวย แต่ไม่ค่อยชอบภายใน,NaN,"[[""ข้างนอก"", ""pos"", ""สวย""],[""ภายใน"", ""neg"", ""ไ...","[[""ข้างนอก"", ""pos"", ""สวย""],[""ภายใน"", ""neg"", """"]]","[[""ภายใน"", ""neg"", [""ไม่ค่อยชอบ""]], [""ภายนอก"", ..."
6,6,คันจริงสวยมากครับ ดูลงตัวดีครับ,NaN,"[[""คันจริง"", ""pos"", ""สวย""], [""คันจริง"", ""pos"",...","[[""คัน"", ""pos"", ""สวย""], [""คัน"", ""pos"", ""ลงตัว""]]","Output: [[""คันจริง"", ""pos"", ""สวยมาก""]]\n\nInst..."
7,7,ใครๆก็อาจจะมองว่าชอบรถคันนี้ดูสวยน่ารักดี แต่...,NaN,"[[""รถคันนี้"", ""neg"", ""ไม่ชอบ""], [""ตาไฟ"",""neg"",...","[[""รถคันนี้"", ""neg"", ""ไม่ชอบหนอนสีๆ"", ""ตาไฟรถเ...",Aspect: รถ\nSentiment: neg\nOpinion words: ไม่...
8,8,ช่องใส่มือถือเหมาะกับให้ฝุ่นลงไปสะสมมากเลยครับ,NaN,"[""ช่องใส่มือถือ"", ""neg"", ""เหมาะกับให้ฝุ่นลงไปส...","[[""ช่องใส่มือถือ"", ""neg"", ""เหมาะกับให้ฝุ่นลงไป...","[(""ช่องใส่มือถือ"", ""neg"", [""ฝุ่น"", ""สะสม""])]"
9,9,ชอบที่สุดคือหน้าปัดเรือนไมล์ ไม่แสบตาแน่ๆ ให้แ...,NaN,"[[""หน้าปัดเรือนไมล์"", ""pos"", [""ชอบที่สุด"",""ไม่...","[[""หน้าปัดเรือนไมล์"", ""pos"", ""ชอบที่สุด""], [""จ...","[[""หน้าปัดเรือนไมล์"", ""pos"", ""ไม่แสบตา""], [""จอ..."
